In [159]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout, BatchNormalization, InputLayer

df = pd.read_csv("Simulation_Outputs_fr", delimiter="\t")
df2=pd.read_csv("consommation_heat_fr")

In [160]:
col_2 = df.iloc[:, 1].values
temp_ext_fr= pd.DataFrame(col_2.reshape(-1, 24))
temp_ext_fr["clusters"]=df2["clusters"]

In [161]:
def normalise(x):
    norm=StandardScaler()
    X_norm=norm.fit_transform(X)
    return X_norm

In [162]:
X=temp_ext_fr.drop(columns=["clusters"])
y=temp_ext_fr["clusters"]
X_scaled=normalise(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4) 

In [163]:


def f1_score(y_true, y_pred):
    precision = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1))) / (tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1))) + tf.keras.backend.epsilon())
    recall = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1))) / (tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1))) + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))



In [164]:
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

model.summary()

C:\Users\toubia\anaconda3\envs\notebook\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_66 (Dense)                     │ (None, 10)                  │             250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 4)                   │              44 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 404 (1.58 KB)

 Trainable params: 404 (1.58 KB)

 Non-trainable params: 0 (0.00 B)

In [165]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_score])

In [166]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.1544 - f1_score: 0.0000e+00 - loss: 1.5228 - val_accuracy: 0.0822 - val_f1_score: 0.0000e+00 - val_loss: 1.4853
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1720 - f1_score: 0.0000e+00 - loss: 1.4429 - val_accuracy: 0.2466 - val_f1_score: 0.0000e+00 - val_loss: 1.4181
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2614 - f1_score: 0.0000e+00 - loss: 1.3702 - val_accuracy: 0.3699 - val_f1_score: 0.0000e+00 - val_loss: 1.3512
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4246 - f1_score: 0.0000e+00 - loss: 1.2992 - val_accuracy: 0.5479 - val_f1_score: 0.0000e+00 - val_loss: 1.2729
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6783 - f1_score: 0.0103 - loss: 1.1959 - val_accuracy: 0.6438 - val_f1_score: 0.2261 - val_loss: 1.2009
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7252 - f1_score: 0.2228 - loss: 1.1160 - val_accuracy: 0.6

In [167]:
model = Sequential()
model.add(Dense(units=120, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=60, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

model.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_69 (Dense)                     │ (None, 120)                 │           3,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 60)                  │           7,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 4)                   │             244 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,504 (41.03 KB)

 Trainable params: 10,504 (41.03 KB)

 Non-trainable params: 0 (0.00 B)

In [168]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_score])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 47ms/step - accuracy: 0.6772 - f1_score: 0.3069 - loss: 1.0614 - val_accuracy: 0.6438 - val_f1_score: 0.4707 - val_loss: 0.9610
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7549 - f1_score: 0.6369 - loss: 0.7318 - val_accuracy: 0.7397 - val_f1_score: 0.5441 - val_loss: 0.8710
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8102 - f1_score: 0.6550 - loss: 0.6774 - val_accuracy: 0.7397 - val_f1_score: 0.6096 - val_loss: 0.7949
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7823 - f1_score: 0.6667 - loss: 0.6553 - val_accuracy: 0.7534 - val_f1_score: 0.6421 - val_loss: 0.7419
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7497 - f1_score: 0.6251 - loss: 0.6707 - val_accuracy: 0.7534 - val_f1_score: 0.6660 - val_loss: 0.7188
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8152 - f1_score: 0.6891 - loss: 0.5800 - val_accuracy: 0.7808 - val_f1_score: 0.6590 - va

In [ ]:
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [119]:
X_train_cnn

array([[[ 0.86981902],
        [ 0.92814425],
        [ 0.98139963],
        ...,
        [ 1.60230879],
        [ 1.54451322],
        [ 1.49085136]],

       [[-1.5336197 ],
        [-1.55604144],
        [-1.57665424],
        ...,
        [-1.02114904],
        [-1.01176651],
        [-1.00021633]],

       [[ 1.20296895],
        [ 1.17329415],
        [ 1.15638052],
        ...,
        [ 1.58835423],
        [ 1.55181687],
        [ 1.50613399]],

       ...,

       [[ 0.79049761],
        [ 0.83008429],
        [ 0.85641329],
        ...,
        [ 1.09296725],
        [ 1.13550846],
        [ 1.18519889]],

       [[-0.83559129],
        [-0.78790507],
        [-0.72674709],
        ...,
        [-0.88160341],
        [-0.88760435],
        [-0.88559665]],

       [[ 0.08453706],
        [-0.02794037],
        [-0.16847475],
        ...,
        [-0.30248905],
        [-0.35443744],
        [-0.41183531]]])

In [126]:
model_cnn = Sequential([
    InputLayer(input_shape=(24, 1)), 
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax') 
])


model_cnn.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy',f1_score])


history_cnn = model_cnn.fit(X_train_cnn, y_train, 
                            validation_data=(X_test_cnn, y_test), 
                            epochs=50, batch_size=32)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.5029 - f1_score: 0.4313 - loss: 1.5800 - val_accuracy: 0.6164 - val_f1_score: 0.0000e+00 - val_loss: 1.2498
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7501 - f1_score: 0.7293 - loss: 0.6457 - val_accuracy: 0.7671 - val_f1_score: 0.0000e+00 - val_loss: 1.1823
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7259 - f1_score: 0.7348 - loss: 0.6047 - val_accuracy: 0.7671 - val_f1_score: 0.1071 - val_loss: 1.1332
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7802 - f1_score: 0.7756 - loss: 0.5454 - val_accuracy: 0.7808 - val_f1_score: 0.2261 - val_loss: 1.0859
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8078 - f1_score: 0.8041 - loss: 0.5485 - val_accuracy: 0.7945 - val_f1_score: 0.3139 - val_loss: 1.0434
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7622 - f1_score: 0.7510 - loss: 0.4988 - val_accuracy: 0.7534 - val_f1_score: 0.36

In [158]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from keras.callbacks import EarlyStopping

df_fr = pd.read_csv("Simulation_Outputs_fr", delimiter="\t", index_col =False)
df_sp = pd.read_csv("Simulation_Outputs_sp", delimiter="\t", index_col =False)
col_2_fr = df_fr.iloc[:, 1].values
col_2_sp = df_sp.iloc[:, 1].values
temp_ext_fr= pd.DataFrame(col_2_fr.reshape(-1, 24))
temp_ext_sp= pd.DataFrame(col_2_sp.reshape(-1, 24))
temp_ext_fr_sp = pd.concat([temp_ext_fr, temp_ext_sp], axis=0).reset_index(drop=True)

X = temp_ext_fr_sp.iloc[:-1, :]  
y = temp_ext_fr_sp.iloc[1:, :]  


X_reshaped = X.values.reshape(X.shape[0], X.shape[1]) 
y_reshaped = y.values.reshape(y.shape[0], y.shape[1])  

X_scaled = normalise(X_reshaped)  
y_scaled = normalise(y_reshaped)  


X_scaled = X_scaled.reshape(X.shape[0], X.shape[1], 1) 
y_scaled = y_scaled.reshape(y.shape[0], y.shape[1], 1) 


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


model = Sequential()
model.add(SimpleRNN(120, activation='tanh', input_shape=(24, 1), return_sequences=True))  
model.add(SimpleRNN(100, activation='tanh', return_sequences=True)) 
model.add(SimpleRNN(80, activation='tanh'))


model.add(Dropout(0.2)) 

model.add(Dense(24, activation='linear'))  

model.compile(optimizer='adam', loss='mean_squared_error',)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")


predictions = model.predict(X_test)
print(predictions)


Epoch 1/100


C:\Users\toubia\anaconda3\envs\notebook\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - loss: 0.7647 - val_loss: 0.2245
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2983 - val_loss: 0.1479
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.2066 - val_loss: 0.1150
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1694 - val_loss: 0.1012
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1582 - val_loss: 0.0982
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1448 - val_loss: 0.0989
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1432 - val_loss: 0.0870
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1336 - val_loss: 0.0831
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1176 - val_loss: 0.0988
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1380 - val_loss: 0.0835
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1122 - val_loss: 0.0773
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1